In [1]:
!apt-get install --no-install-recommends -y openmpi-bin libopenmpi-dev libopencv-dev python-opencv && ln -sf /usr/lib/x86_64-linux-gnu/libmpi_cxx.so /usr/lib/x86_64-linux-gnu/libmpi_cxx.so.1 && ln -sf /usr/lib/x86_64-linux-gnu/openmpi/lib/libmpi.so /usr/lib/x86_64-linux-gnu/openmpi/lib/libmpi.so.12 && ln -sf /usr/lib/x86_64-linux-gnu/libmpi.so /usr/lib/x86_64-linux-gnu/libmpi.so.12 && pip install cntk

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  gir1.2-atk-1.0 gir1.2-freedesktop gir1.2-gdkpixbuf-2.0 gir1.2-glib-2.0
  gir1.2-gtk-2.0 gir1.2-pango-1.0 icu-devtools libatk1.0-dev libavcodec-dev
  libavcodec57 libavformat-dev libavformat57 libavutil-dev libavutil55
  libbluray1 libcairo-script-interpreter2 libcairo2-dev libchromaprint1
  libcrystalhd3 libcv-dev libcvaux-dev libdc1394-22 libdc1394-22-dev libexpat1
  libexpat1-dev libfabric1 libfontconfig1-dev libgdk-pixbuf2.0-dev
  libgirepository-1.0-1 libglib2.0-bin libglib2.0-dev libgme0 libgraphite2-dev
  libgsm1 libgtk2.0-dev libharfbuzz-dev libharfbuzz-gobject0 libharfbuzz-icu0
  libhighgui-dev libhwloc-dev libhwloc-plugins libhwloc5 libibverbs-dev
  libibverbs1 libicu-dev libicu57 libilmbase-dev libjbig-dev libjpeg-dev
  libjpeg62-turbo-dev libltdl-dev liblzma-dev libmp3lame0 libmpg123-0
  libnl-3-200 libnl-route-3-200 libnuma-d

Get:21 http://deb.debian.org/debian stretch/main amd64 libglib2.0-dev amd64 2.50.3-2+deb9u1 [2983 kB]
Get:22 http://deb.debian.org/debian stretch/main amd64 libatk1.0-dev amd64 2.22.0-1 [115 kB]
Get:23 http://deb.debian.org/debian stretch/main amd64 libva1 amd64 1.7.3-2 [52.1 kB]
Get:24 http://deb.debian.org/debian stretch/main amd64 libva-drm1 amd64 1.7.3-2 [15.0 kB]
Get:25 http://deb.debian.org/debian stretch/main amd64 libva-x11-1 amd64 1.7.3-2 [19.5 kB]
Get:26 http://deb.debian.org/debian stretch/main amd64 libvdpau1 amd64 1.1.1-6 [41.2 kB]
Get:27 http://deb.debian.org/debian stretch/main amd64 libavutil55 amd64 7:3.2.14-1~deb9u1 [221 kB]
Get:28 http://deb.debian.org/debian stretch/main amd64 libcrystalhd3 amd64 1:0.0~git20110715.fdd2f19-12 [45.8 kB]
Get:29 http://deb.debian.org/debian stretch/main amd64 libgsm1 amd64 1.0.13-4+b2 [24.3 kB]
Get:30 http://deb.debian.org/debian stretch/main amd64 libmp3lame0 amd64 3.99.5+repack1-9+b2 [354 kB]
Get:31 http://deb.debian.org/debian stretc

Get:114 http://deb.debian.org/debian stretch/main amd64 libtiffxx5 amd64 4.0.8-2+deb9u4 [95.8 kB]
Get:115 http://deb.debian.org/debian stretch/main amd64 libtiff5-dev amd64 4.0.8-2+deb9u4 [361 kB]
Get:116 http://deb.debian.org/debian stretch/main amd64 libraw1394-dev amd64 2.1.2-1+b1 [57.9 kB]
Get:117 http://deb.debian.org/debian stretch/main amd64 libdc1394-22-dev amd64 2.2.5-1 [138 kB]
Get:118 http://deb.debian.org/debian stretch/main amd64 libopencv-highgui-dev amd64 2.4.9.1+dfsg1-2 [129 kB]
Get:119 http://deb.debian.org/debian stretch/main amd64 libopencv-objdetect2.4v5 amd64 2.4.9.1+dfsg1-2 [192 kB]
Get:120 http://deb.debian.org/debian stretch/main amd64 libopencv-objdetect-dev amd64 2.4.9.1+dfsg1-2 [215 kB]
Get:121 http://deb.debian.org/debian stretch/main amd64 libopencv-video2.4v5 amd64 2.4.9.1+dfsg1-2 [121 kB]
Get:122 http://deb.debian.org/debian stretch/main amd64 libopencv-video-dev amd64 2.4.9.1+dfsg1-2 [136 kB]
Get:123 http://deb.debian.org/debian stretch/main amd64 libope

Unpacking libglib2.0-bin (2.50.3-2+deb9u1) ...
Selecting previously unselected package libpcre16-3:amd64.
Preparing to unpack .../014-libpcre16-3_2%3a8.39-3_amd64.deb ...
Unpacking libpcre16-3:amd64 (2:8.39-3) ...
Selecting previously unselected package libpcre32-3:amd64.
Preparing to unpack .../015-libpcre32-3_2%3a8.39-3_amd64.deb ...
Unpacking libpcre32-3:amd64 (2:8.39-3) ...
Selecting previously unselected package libpcre3-dev:amd64.
Preparing to unpack .../016-libpcre3-dev_2%3a8.39-3_amd64.deb ...
Unpacking libpcre3-dev:amd64 (2:8.39-3) ...
Selecting previously unselected package libglib2.0-dev.
Preparing to unpack .../017-libglib2.0-dev_2.50.3-2+deb9u1_amd64.deb ...
Unpacking libglib2.0-dev (2.50.3-2+deb9u1) ...
Selecting previously unselected package libatk1.0-dev:amd64.
Preparing to unpack .../018-libatk1.0-dev_2.22.0-1_amd64.deb ...
Unpacking libatk1.0-dev:amd64 (2.22.0-1) ...
Selecting previously unselected package libva1:amd64.
Preparing to unpack .../019-libva1_1.7.3-2_amd64

Selecting previously unselected package libxrender-dev:amd64.
Preparing to unpack .../061-libxrender-dev_1%3a0.9.10-1_amd64.deb ...
Unpacking libxrender-dev:amd64 (1:0.9.10-1) ...
Selecting previously unselected package x11proto-xext-dev.
Preparing to unpack .../062-x11proto-xext-dev_7.3.0-1_all.deb ...
Unpacking x11proto-xext-dev (7.3.0-1) ...
Selecting previously unselected package libxext-dev:amd64.
Preparing to unpack .../063-libxext-dev_2%3a1.3.3-1+b2_amd64.deb ...
Unpacking libxext-dev:amd64 (2:1.3.3-1+b2) ...
Selecting previously unselected package libpixman-1-dev.
Preparing to unpack .../064-libpixman-1-dev_0.34.0-1_amd64.deb ...
Unpacking libpixman-1-dev (0.34.0-1) ...
Selecting previously unselected package libxcb-render0-dev:amd64.
Preparing to unpack .../065-libxcb-render0-dev_1.12-1_amd64.deb ...
Unpacking libxcb-render0-dev:amd64 (1.12-1) ...
Selecting previously unselected package libxcb-shm0-dev:amd64.
Preparing to unpack .../066-libxcb-shm0-dev_1.12-1_amd64.deb ...
Unp

Unpacking x11proto-damage-dev (1:1.2.1-2) ...
Selecting previously unselected package libxdamage-dev:amd64.
Preparing to unpack .../106-libxdamage-dev_1%3a1.1.4-2+b3_amd64.deb ...
Unpacking libxdamage-dev:amd64 (1:1.1.4-2+b3) ...
Selecting previously unselected package libxml2-utils.
Preparing to unpack .../107-libxml2-utils_2.9.4+dfsg1-2.2+deb9u2_amd64.deb ...
Unpacking libxml2-utils (2.9.4+dfsg1-2.2+deb9u2) ...
Selecting previously unselected package libgtk2.0-dev.
Preparing to unpack .../108-libgtk2.0-dev_2.24.31-2_amd64.deb ...
Unpacking libgtk2.0-dev (2.24.31-2) ...
Selecting previously unselected package libjpeg62-turbo-dev:amd64.
Preparing to unpack .../109-libjpeg62-turbo-dev_1%3a1.5.1-2_amd64.deb ...
Unpacking libjpeg62-turbo-dev:amd64 (1:1.5.1-2) ...
Selecting previously unselected package libjpeg-dev.
Preparing to unpack .../110-libjpeg-dev_1%3a1.5.1-2_all.deb ...
Unpacking libjpeg-dev (1:1.5.1-2) ...
Selecting previously unselected package libjbig-dev:amd64.
Preparing to un

Selecting previously unselected package libopencv-superres2.4v5:amd64.
Preparing to unpack .../149-libopencv-superres2.4v5_2.4.9.1+dfsg1-2_amd64.deb ...
Unpacking libopencv-superres2.4v5:amd64 (2.4.9.1+dfsg1-2) ...
Selecting previously unselected package libopencv-superres-dev:amd64.
Preparing to unpack .../150-libopencv-superres-dev_2.4.9.1+dfsg1-2_amd64.deb ...
Unpacking libopencv-superres-dev:amd64 (2.4.9.1+dfsg1-2) ...
Selecting previously unselected package libopencv-ocl-dev:amd64.
Preparing to unpack .../151-libopencv-ocl-dev_2.4.9.1+dfsg1-2_amd64.deb ...
Unpacking libopencv-ocl-dev:amd64 (2.4.9.1+dfsg1-2) ...
Selecting previously unselected package libopencv2.4-jni.
Preparing to unpack .../152-libopencv2.4-jni_2.4.9.1+dfsg1-2_amd64.deb ...
Unpacking libopencv2.4-jni (2.4.9.1+dfsg1-2) ...
Selecting previously unselected package libopencv2.4-java.
Preparing to unpack .../153-libopencv2.4-java_2.4.9.1+dfsg1-2_all.deb ...
Unpacking libopencv2.4-java (2.4.9.1+dfsg1-2) ...
Selecting p

Setting up libxft-dev (2.3.2-1+b2) ...
Setting up libgdk-pixbuf2.0-dev (2.36.5-2+deb9u2) ...
Setting up libjpeg-dev (1:1.5.1-2) ...
Setting up libswscale4:amd64 (7:3.2.14-1~deb9u1) ...
Setting up libopencv-ml-dev:amd64 (2.4.9.1+dfsg1-2) ...
Setting up libopencv-photo2.4v5:amd64 (2.4.9.1+dfsg1-2) ...
Setting up libopencv-video2.4v5:amd64 (2.4.9.1+dfsg1-2) ...
Setting up libopencv-video-dev:amd64 (2.4.9.1+dfsg1-2) ...
Setting up libxfixes-dev:amd64 (1:5.0.3-1) ...
Setting up libharfbuzz-dev:amd64 (1.4.2-1) ...
Setting up x11proto-composite-dev (1:0.4.2-2) ...
Setting up gir1.2-gtk-2.0 (2.24.31-2) ...
Setting up libibverbs1 (1.2.1-2) ...
Setting up libavutil-dev:amd64 (7:3.2.14-1~deb9u1) ...
Setting up libxcursor-dev:amd64 (1:1.1.14-1+deb9u2) ...
Setting up libopencv-photo-dev:amd64 (2.4.9.1+dfsg1-2) ...
Setting up libxrandr-dev:amd64 (2:1.5.1-1) ...
Setting up libcairo2-dev (1.14.8-1) ...
Setting up libxcomposite-dev:amd64 (1:0.4.4-2) ...
Setting up libdc1394-22-dev:amd64 (2.2.5-1) ...
S

In [7]:
import cntk as C

model_path = '/kaggle/input/pretrained-deepdanboru/model.cntk'

ctnk_model = C.load_model(model_path)
ctnk_model.save('./model.onnx', format=C.ModelFormat.ONNX)

In [3]:
!pip install onnxruntime

     |████████████████████████████████| 3.6MB 5.9MB/s eta 0:00:01


In [9]:
import onnxruntime
import numpy as np

In [27]:
ort_session = onnxruntime.InferenceSession("./model.onnx")

x = np.random.rand(1, 3, 299, 299).astype(np.float32)
# compute ONNX Runtime output prediction
ort_inputs = {ort_session.get_inputs()[0].name: x}
ort_outs = ort_session.run(None, ort_inputs)

# compute the ctnk output
cntk_out = ctnk_model.eval(x)

In [28]:
np.testing.assert_allclose(np.array(cntk_out), ort_outs[0], rtol=1e-03, atol=1e-05)

AssertionError: 
Not equal to tolerance rtol=0.001, atol=1e-05

Mismatched elements: 564 / 6681 (8.44%)
Max absolute difference: 0.07434976
Max relative difference: 1.2049851
 x: array([[1.199393e-06, 9.261483e-03, 2.261244e-01, ..., 9.516850e-01,
        2.648727e-01, 1.228109e-02]], dtype=float32)
 y: array([[1.162291e-06, 9.180605e-03, 1.849605e-01, ..., 9.503207e-01,
        2.576922e-01, 1.243669e-02]], dtype=float32)